In [1]:
pip install kiwipiepy #형태소 분석기
pip install tomotopy #토픽 추출기

pip install pyvis #이거는 아래 보고와서 깔도록, CTM에서 쓰임돠

SyntaxError: invalid syntax (<ipython-input-1-8171591eedde>, line 1)

In [14]:
import sys
import io
import re # 정규표현식 패키지
from kiwipiepy import Kiwi, Option
import tomotopy as tp

filename="test.txt"  #파일 경로
kiwi=Kiwi()
kiwi.prepare()
stopwords = set(["명", "어제", "등", "이","하","돠","있","일","것","들","적","그","씨","되","없","다","이다"])


def tokenize(sent): # 파일의 라인을 분석할 tokenize 함수
    res, score = kiwi.analyze(sent)[0] # 첫번째 결과를 사용한다, 분석할때 나오는 결과에서 단어만 추출
    return [word
            for word, tag, _, _ in res
            if tag.startswith('N') and word not in stopwords] #불용어사전 적용

#N으로 시작하는 NNG (일반명사), NNP(고유명사), NNB(의존명사) NR(수사), NP(대명사)가 있다

#pat_tag = re.compile('NN[GP]')
#def tokenizer(raw, user_data):
#        res, _ = user_data()[0]
#        for w, tag, start, l in res:
#            if not pat_tag.match(tag) or len(w) <= 1: continue
#            yield w + ('다' if tag.startswith('V') else ''), start, l

#위의 코드와 같이 정규표현식을 통해 원하는 형태로 추출 가능 일단은 ,명사위주로 해놨다.
            

model = tp.LDAModel(k=2, alpha=0.1, eta=0.01, min_cf=20,min_df=10, tw=tp.TermWeight.PMI)
#LDA 모델을 적용해서 토픽 추출, k는 topic 개수
#alpha는 문헌-토픽 디리클레 분포의 하이퍼 파라미터
#eta는 토픽-단어 디리클레 분포의 하이퍼 파라미터 두개다 상수인듯하다
#min_cf는 단어의 최소 장서 빈도수, 전체 문헌내 출현빈도
#min_df는 단어의 최서 문헌 빈도수, 출현한 문헌 숫자수 의미
#tw는 용어 가중치 기법으로, ONE, IDF, PMI를 사용가능, ONE 보다는 PMI나 IDF 둘중 하나 사용 


for i, line in enumerate(open(filename, encoding='utf-8')): #해당 경로의 파일을 받아와 한 라인씩 model에 추가
    print(tokenize(line))
    model.add_doc(tokenize(line)) #추출하고 모델안의 문헌을 넣는다. 즉, 학습과정에 쓰일 문헌을 생성
    if i % 10 == 0: print('Document #{} has been loaded'.format(i))
 
model.train(0) #학습 초기화

print('Total docs:', len(model.docs)) #문헌들
print('Total words:', model.num_words) #전체단어개수
print('Vocab size:', model.num_vocabs) #전체 어휘개수
 
 
for i in range(0,2):
    print('Iteration {}\tLL per word: {}'.format(i, model.ll_per_word))
    model.train(1) #문헌 학습, 안에 숫자는 깁스 샘플링의 반복횟수
		#이때 기본값으로 시스템내 가용한 모든 스레드의 개수사용, 그리고 병렬화 방법을 찾아서 실행시켜준다
  
for i in range(model.k): #k는 토픽의 개수
    res = model.get_topic_words(i, top_n=5) #하위 토픽 i에 해당하는 top_n개의 단어 반환
    print('Topic #{}'.format(i), end='\t')
    print(', '.join(w for w, p in res))

['신년', '인터뷰', '사회', '평등', '연구', '대가', '로버트', '라이', '시', '캘리포니아', '대학', '교수', '중산', '소득', '결국', '밑', '지속', '가능', '경제', '성장', '기대', '수', '정부', '무엇', '계층', '이동', '사다리', '구실', '교육', '투자', '사회', '평등', '연구', '분야', '세계', '석학', '로버트', '라이', '시', '미국', '캘리포니아', '대학', '버클리', '캠퍼스', '공공정책', '대학원', '교수', '서울신문', '인터뷰', '양극', '해법', '공정', '교육', '상위', '부', '독점', '정치', '경제', '구조', '자신', '유리', '상황', '교육', '계층', '이동', '기회', '의미', '현', '미국', '사회', '라이', '시', '교수', '비판', '그간', '양극', '신음', '지구촌', '곳곳', '관심', '서울', '신문', '년', '인터뷰', '진행', '이유', '아래', '일문일답', '로버트', '라이', '시', '미국', '캘리포니아', '대', '공공정책', '대학원', '교수', '서울신문', '인터뷰', '세계', '경제', '중국', '미국', '식', '제로', '섬', '게임', '인식', '도널드', '트럼프', '미국', '대통령', '비판', '세계', '경제', '유기', '연결', '때문', '미국', '우선주의', '나라', '위협', '뿐', '결국', '미국', '자신', '위험', '로버트', '라이', '시', '교수', '제공', '세계', '경제', '양극', '경제', '양극', '사회', '상위', '계층', '부', '독점', '심화', '미국', '경우', '상위', '정치', '경제', '정책', '자신', '유리', '양극', '소외', '좌절', '정치', '부패', '도널드', '트럼프', '대통령', '분노', '소수자', '이민자',

In [5]:
import re # 정규표현식 패키지
import tomotopy as tp # 토픽 모델링에 사용할 패키지
from kiwipiepy import Kiwi # 한국어 형태소 분석에 사용할 패키지
from pyvis.network import Network # 네트워크 시각화에 사용할 패키지
 
try:
    # 이미 전처리된 코퍼스가 있으면 불러온다.
    corpus = tp.utils.Corpus.load('k.cps')
except IOError:
    # 없으면 전처리를 시작한다.
    kiwi = Kiwi()
    kiwi.prepare()
 
    # 형태소 분석 후 사용할 태그, 원하는대로 넣을 수 있다.
    """pat_tag = re.compile('NN[GP]')"""
    pat_tag = re.compile('NN[GP]|V[VA]|MAG|MM')
 
    def tokenizer(raw, user_data):
        res, _ = user_data()[0]
        for w, tag, start, l in res:
            if not pat_tag.match(tag) or len(w) <= 1: continue
            yield w + ('다' if tag.startswith('V') else ''), start, l
 
    corpus = tp.utils.Corpus(
        tokenizer=tokenizer
    )
    # 입력 파일에는 한 줄에 문헌 하나씩이 들어가 있습니다.
    corpus.process((line, kiwi.async_analyze(line)) for line in open('nlptesting/document/datas.txt', 'r',-1,'utf-8'))
    # 전처리한 코퍼스를 저장한다.
    corpus.save('k.cps')
 
# 최소 10개 이상 문헌에 등장하고, 전체 출현빈도는 20 이상인 단어만 사용합니다.
# 그리고 상위 10개 고빈도 단어는 분석에서 제외하구요
# 주제 개수는 40개입니다.
mdl = tp.CTModel(tw=tp.TermWeight.ONE, min_df=10, min_cf=20, k=10, corpus=corpus)
mdl.train(0)
 
# 문헌 수가 만 개 이상이라면 num_beta_sample을 1~5 정도로 줄여도 됨
# 수 천개라면 최소 10 정도, 수 백개에 불과하다면 20 이상으로 키우는걸 권장
mdl.num_beta_sample = 1
print('Num docs:{}, Num Vocabs:{}, Total Words:{}'.format(
    len(mdl.docs), len(mdl.used_vocabs), mdl.num_words
))
print('Removed Top words: ', *mdl.removed_top_words)
 
# 총 1000회 깁스샘플링을 반복합니다.
# 적절한 반복회수는 데이터에 따라 다릅니다.
# ll_per_word 값의 증가가 둔화되거나 멈추는 지점까지만
# 반복하는걸 추천합니다.
for i in range(0, 1000, 20):
    print('Iteration: {:04}, LL per word: {:.4}'.format(i, mdl.ll_per_word))
    mdl.train(20)
print('Iteration: {:04}, LL per word: {:.4}'.format(i, mdl.ll_per_word))

for i in range(mdl.k):
    res = mdl.get_topic_words(i, top_n=5)
    print('Topic #{}'.format(i), end='\t')
    print(', '.join(w for w, p in res))

 # 학습된 결과를 시각화 합니다.
g = Network(width=800, height=800, font_color="#333")
correl = mdl.get_correlations().reshape([-1])
correl.sort()
 
# 상관계수 상위 10%만 간선으로 잇습니다.
top_tenth = mdl.k * (mdl.k - 1) // 10
top_tenth = correl[-mdl.k - top_tenth]
 
topic_counts = mdl.get_count_by_topics()
 
for k in range(mdl.k):
    label = "#{}".format(k)
    title= ' '.join(word for word, _ in mdl.get_topic_words(k, top_n=8))
    print('Topic', label, title)
    label += '\n' + ' '.join(word for word, _ in mdl.get_topic_words(k, top_n=3))
    g.add_node(k, label=label, title=title, shape='ellipse', value=float(topic_counts[k]))
    for l, correlation in zip(range(k - 1), mdl.get_correlations(k)):
        if correlation < top_tenth: continue
        g.add_edge(k, l, value=float(correlation), title='{:.02}'.format(correlation))
 
g.barnes_hut(gravity=-1000, spring_length=20)
g.show_buttons()
# 시각화 파일이 topic_network.html 이라는 이름으로 저장됩니다.
# 웹 브라우저로 열어서 확인해보세요.
g.show("topic_network.html")

FileNotFoundError: [Errno 2] No such file or directory: 'nlptesting/document/datas.txt'